In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../../data/training.csv')
df.head()

,txkey,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,...,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk,label
0,1c09727c939eb69ead2a4ce4072b8aa18992a64f01fcb4...,46,101812,84d2dc85d4da6a7fa284a11a4290d7e9a969163dcb4d82...,3dd5bf1e29e5e0baa789ce692fe5dbd34ff05173acf351...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,375.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,1529,0,0
1,2043f245a93bc6328dac964d6dbc89f13a0346062c194d...,17,85509,9a8cf5d0afd729cb7876f6b3172152c7c9c6fabd40515c...,8cb13f9b38c7bbc02d210e580dcbbcbb6c95bf18bc3320...,5,8.0,4356c6642ef6e837543b577c7ee3ffa92b4b8fcfb57254...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,282.0,...,0.0,15759.0,NaN,0,0,8.0,70.0,101,0,0
2,e4853710290a8409279f3279f8032ae670824bd19aa173...,44,212641,dcc1389a5765d6f53152cf85970fbe78a83fd3d1c299b0...,1ec32868e5e1d5ff8df56737c2a91326cbfe3364382de6...,5,4.0,5b7eff061f8896aac4339ea35c25f8bb956a43bc486460...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,288.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,116,0,0
3,74d811b1dbc28b22b73ba2c79bb6033791d913b6d27a25...,42,102702,577f2329d1eccd59ba0abaf6113bb78dcd575badcbc57f...,4359dca1ac6a835eceb2bc0dd6b0b710f030c3499126e9...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,36684976be1f529e6e2a32c9edab4cf8e364b2b916ae2c...,375.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,1683,0,0
4,68ca182343969d429d79a34e532bc1ca7a3cc032c2ad81...,31,185737,fff6b4126c40620b1fbb11d4de02cd67b9e95071caa40b...,a3837f2905383f235a72679482c5f02e40f2a8ca29750d...,5,5.0,50d5b02ce3fc88723438c2a29cfdb04be4a1a11280ddb6...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,406.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,14,0,0


In [4]:
df['label'].value_counts()

label
0    8656497
1      32029
Name: count, dtype: int64

# locdt loctm
Normalize loctm, for example 120000 => 0.5

In [5]:
df['normalized_loctm'] = df['loctm'] // 10000 * 3600 + (df['loctm'] % 10000)//100 * 60 + df['loctm'] % 100
scaler = MinMaxScaler()
df['normalized_loctm'] = scaler.fit_transform(df['normalized_loctm'].values.reshape(-1, 1))
#df['normalized_loctm']=(df['normalized_loctm']-df['normalized_loctm'].min())/(df['normalized_loctm'].max()-df['normalized_loctm'].min())

In [6]:
df[df['loctm']==71232]['normalized_loctm']

159903     0.300374
280986     0.300374
796157     0.300374
842496     0.300374
929400     0.300374
1008015    0.300374
1095361    0.300374
1119909    0.300374
1245997    0.300374
1473137    0.300374
2045789    0.300374
2152061    0.300374
2336443    0.300374
2493239    0.300374
2750654    0.300374
2775286    0.300374
3011176    0.300374
3041116    0.300374
3049547    0.300374
3078384    0.300374
3167224    0.300374
3181486    0.300374
3239197    0.300374
3528567    0.300374
3607408    0.300374
3676503    0.300374
4203422    0.300374
4222355    0.300374
4384338    0.300374
4660831    0.300374
5134638    0.300374
5247071    0.300374
5583921    0.300374
5648670    0.300374
5798173    0.300374
5835314    0.300374
5852533    0.300374
6060409    0.300374
6079382    0.300374
6158772    0.300374
6246404    0.300374
6443213    0.300374
6446069    0.300374
6493078    0.300374
6763543    0.300374
7133801    0.300374
7720996    0.300374
7913468    0.300374
8297658    0.300374
8379551    0.300374


In [7]:
normalized_loctm = df['normalized_loctm']
df.drop(['loctm', 'normalized_loctm'], axis=1, inplace=True)
df.insert(1, 'loctm', normalized_loctm)
df.head()

,txkey,loctm,locdt,chid,cano,contp,etymd,mchno,acqic,mcc,...,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk,label
0,1c09727c939eb69ead2a4ce4072b8aa18992a64f01fcb4...,0.429311,46,84d2dc85d4da6a7fa284a11a4290d7e9a969163dcb4d82...,3dd5bf1e29e5e0baa789ce692fe5dbd34ff05173acf351...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,375.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,1529,0,0
1,2043f245a93bc6328dac964d6dbc89f13a0346062c194d...,0.371636,17,9a8cf5d0afd729cb7876f6b3172152c7c9c6fabd40515c...,8cb13f9b38c7bbc02d210e580dcbbcbb6c95bf18bc3320...,5,8.0,4356c6642ef6e837543b577c7ee3ffa92b4b8fcfb57254...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,282.0,...,0.0,15759.0,NaN,0,0,8.0,70.0,101,0,0
2,e4853710290a8409279f3279f8032ae670824bd19aa173...,0.893540,44,dcc1389a5765d6f53152cf85970fbe78a83fd3d1c299b0...,1ec32868e5e1d5ff8df56737c2a91326cbfe3364382de6...,5,4.0,5b7eff061f8896aac4339ea35c25f8bb956a43bc486460...,8f6b3ff512a001e0d1988c6cd888ef8c74112fb71117e5...,288.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,116,0,0
3,74d811b1dbc28b22b73ba2c79bb6033791d913b6d27a25...,0.435445,42,577f2329d1eccd59ba0abaf6113bb78dcd575badcbc57f...,4359dca1ac6a835eceb2bc0dd6b0b710f030c3499126e9...,5,1.0,cad752c5d05d2bdcc30d64fa4e68404c2d1f7be5d14d52...,36684976be1f529e6e2a32c9edab4cf8e364b2b916ae2c...,375.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,1683,0,0
4,68ca182343969d429d79a34e532bc1ca7a3cc032c2ad81...,0.790021,31,fff6b4126c40620b1fbb11d4de02cd67b9e95071caa40b...,a3837f2905383f235a72679482c5f02e40f2a8ca29750d...,5,5.0,50d5b02ce3fc88723438c2a29cfdb04be4a1a11280ddb6...,379166ff4a62dac343b4b734188aa618716cc496e48b65...,406.0,...,0.0,15759.0,NaN,0,0,6.0,70.0,14,0,0


# Original

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold


print('No Frauds', round(df['label'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['label'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

X = df.drop('label', axis=1)
y = df['label']

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]



# See if both the train and test label distribution are similarly distributed
# train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
# test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
# print('-' * 100)

# print('Label Distributions: \n')
# print(train_counts_label/ len(original_ytrain))
# print(test_counts_label/ len(original_ytest))

No Frauds 99.63 % of the dataset
Frauds 0.37 % of the dataset
Train: [1722669 1722851 1722891 ... 8688523 8688524 8688525] Test: [      0       1       2 ... 1737758 1737759 1737760]
Train: [      0       1       2 ... 8688523 8688524 8688525] Test: [1722669 1722851 1722891 ... 3475467 3475468 3475469]
Train: [      0       1       2 ... 8688523 8688524 8688525] Test: [3461625 3461631 3461936 ... 5213249 5213250 5213251]
Train: [      0       1       2 ... 8688523 8688524 8688525] Test: [5175924 5176029 5176360 ... 6950945 6950946 6950947]
Train: [      0       1       2 ... 6950945 6950946 6950947] Test: [6919499 6919679 6919898 ... 8688523 8688524 8688525]


In [9]:
# from sklearn.utils import resample

# original_data = pd.concat([original_Xtrain,  original_ytrain], axis=1)
# min_class_size = len(original_data[original_data['label']==1])

# maj_class = original_data[original_data['label']==0]
# min_class = original_data[original_data['label']==1]

# min_samples = int(1000000 * (min_class_size/len(original_data)))
# maj_samples = 1000000 - min_samples

# maj = resample(maj_class, replace=False, n_samples=maj_samples, random_state=42)
# min = resample(min_class, replace=False, n_samples=min_samples, random_state=42)

# stratified = pd.concat([maj, min])
# original_Xtrain = stratified.drop('label', axis=1)
# original_ytrain = stratified['label']


# Transform object values

In [10]:
cats = ['txkey', 'chid', 'cano', 'mchno', 'acqic']

ord_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoded_data = ord_encoder.fit_transform(original_Xtrain[cats])

original_Xtrain[cats] = encoded_data

/tmp/ipykernel_13174/199024846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_Xtrain[cats] = encoded_data


In [11]:
original_Xtrain.dtypes

txkey        float64
loctm        float64
locdt          int64
chid         float64
cano         float64
contp          int64
etymd        float64
mchno        float64
acqic        float64
mcc          float64
conam        float64
ecfg           int64
insfg          int64
iterm        float64
bnsfg          int64
flam1          int64
stocn        float64
scity        float64
stscd        float64
ovrlt          int64
flbmk          int64
hcefg        float64
csmcu        float64
csmam          int64
flg_3dsmk      int64
dtype: object

# Fill nan values

In [12]:
na = ['etymd', 'mcc', 'stocn', 'scity', 'stscd', 'hcefg', 'csmcu']

simple_imputer = SimpleImputer(strategy="most_frequent")
imputed_data = simple_imputer.fit_transform(original_Xtrain[na])

original_Xtrain[na] = imputed_data

/tmp/ipykernel_13174/3629092023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_Xtrain[na] = imputed_data


In [13]:
original_Xtrain.isna().sum()

txkey        0
loctm        0
locdt        0
chid         0
cano         0
contp        0
etymd        0
mchno        0
acqic        0
mcc          0
conam        0
ecfg         0
insfg        0
iterm        0
bnsfg        0
flam1        0
stocn        0
scity        0
stscd        0
ovrlt        0
flbmk        0
hcefg        0
csmcu        0
csmam        0
flg_3dsmk    0
dtype: int64

# Feature selection
### numeric features: 
locdt, loctm, conam, iterm, flam1, csmam

## First see the relationship between 3 money-related features

## Summary
* conma and flam1 are highly correlated so we choose one

In [14]:
original_Xtrain[['conam', 'flam1', 'csmam']].corr()

,conam,flam1,csmam
conam,1.000000,0.999967,0.099667
flam1,0.999967,1.000000,0.099666
csmam,0.099667,0.099666,1.000000


In [15]:
num_attribs = ['locdt', 'loctm', 'conam', 'iterm', 'csmam']

### categorical features
chid, cano, contp, etymd, mchno, acqic, mcc, ecfg, insfg, bnsfg, stocn, scity, stscd, ovrlt, flbmk, hcefg, csmcu, flg_3dsmk

## See if stocn and scity are correlated
### Summary
* p-value < 0.05
* 

In [16]:
from sklearn.feature_selection import chi2

chi2(original_Xtrain['stocn'].values.reshape(-1, 1), original_Xtrain['scity'])

(array([5.77774482e+08]), array([0.]))

In [17]:
# country_city = pd.crosstab(original_Xtrain['stocn'], original_Xtrain['scity'])

# sns.heatmap(country_city, annot=True, fmt='d', cmap='YlGnBu', cbar=False)
# plt.xlabel('City')
# plt.ylabel('Country')
# plt.title('City Distribution by Country')
# plt.show()

In [18]:
from sklearn.feature_selection import SelectKBest


cat_attribs = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic', 'mcc', 'ecfg', 'insfg', 'bnsfg', 'scity', 'stscd', 
               'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']
cat_original_Xtrain = original_Xtrain[cat_attribs]
selector = SelectKBest(chi2, k=len(cat_attribs)//2)
selector.fit(cat_original_Xtrain, original_ytrain)

SelectKBest(k=8, score_func=<function chi2 at 0x7f3bfc8be200>)

In [19]:
cols_idxs = selector.get_support(indices=True)
new_cat_original_Xtrain = cat_original_Xtrain.iloc[:,cols_idxs]

In [20]:
selected_original_Xtrain = pd.concat([new_cat_original_Xtrain, original_Xtrain[num_attribs]], axis=1)
selected_columns = selected_original_Xtrain.columns
selected_columns

Index(['chid', 'cano', 'mchno', 'acqic', 'mcc', 'ecfg', 'scity', 'csmcu',
       'locdt', 'loctm', 'conam', 'iterm', 'csmam'],
      dtype='object')

In [22]:
[col for col in df.columns if col not in selected_columns]

['txkey',
 'contp',
 'etymd',
 'insfg',
 'bnsfg',
 'flam1',
 'stocn',
 'stscd',
 'ovrlt',
 'flbmk',
 'hcefg',
 'flg_3dsmk',
 'label']

# Smote

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))

# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_

selected_original_Xtrain = selected_original_Xtrain.values
selected_original_Xtest = original_Xtest[selected_columns].values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values



Length of X (train): 1000000 | Length of y (train): 1000000
Length of X (test): 1737705 | Length of y (test): 1737705


In [ ]:
# knears_neighbors_sm = KNeighborsClassifier(n_jobs=-1)
# knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

# rand_knears_neighbors = RandomizedSearchCV(KNeighborsClassifier(), knears_params, n_iter=6)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

log_reg_params = {"penalty": ['l2'], 'C': [0.01, 0.1, 1, 10, 100]}

# rand_log_reg = RandomizedSearchCV(LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1), log_reg_params, n_iter=3, n_jobs=-1)
log_reg = LogisticRegression(max_iter=5000, solver='saga', n_jobs=-1)

robust_scaler = RobustScaler()


for index, (train, test) in enumerate(sss.split(selected_original_Xtrain, original_ytrain)):
    print(index)
    pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy=0.5, k_neighbors=3), log_reg) # SMOTE happens during Cross Validation not before..
    X_train = robust_scaler.fit_transform(selected_original_Xtrain[train])
    model = pipeline.fit(X_train, original_ytrain[train])

    X_test = robust_scaler.transform(selected_original_Xtrain[test])
    prediction = log_reg.predict(X_test)
    
    accuracy_lst.append(pipeline.score(selected_original_Xtrain[test], original_ytrain[test]))
    precision_lst.append(precision_score(original_ytrain[test], prediction))
    recall_lst.append(recall_score(original_ytrain[test], prediction))
    f1_lst.append(f1_score(original_ytrain[test], prediction))
    auc_lst.append(roc_auc_score(original_ytrain[test], prediction))

 

0


In [24]:
from joblib import dump, load
dump(log_reg, 'log_reg_v1.joblib') 

['log_reg_v1.joblib']

In [23]:
print(f'accuracy {np.mean(accuracy_lst)}')
print(f'precision {np.mean(precision_lst)}')
print(f'recall {np.mean(recall_lst)}')
print(f'f1 {np.mean(f1_lst)}')
print(f'auc {np.mean(auc_lst)}')

accuracy 0.67549
precision 0.019879553565530818
recall 0.7360257103249459
f1 0.038712490710340296
auc 0.8008778952180711


In [25]:
to_write = [f'accuracy {np.mean(accuracy_lst)}', f'precision {np.mean(precision_lst)}', f'recall {np.mean(recall_lst)}', f'f1 {np.mean(f1_lst)}', f'auc {np.mean(auc_lst)}']

In [26]:
with open('trained_models/log_reg_v1.txt', 'a') as f:
    for w in to_write:
        f.write(w+'\n')
    